In [6]:
#from sys import argv

In [18]:
#input_genome = argv[1]
#enzyme_name = argv[2]
#output_file = argv[3]
strand = '+'
input_file = 'sample_bad.bed'
output_file = 'sort_bed_plus.bed'

In [19]:
df = open(input_file, 'r')
to_file = open(output_file, 'w')

for i in df:
    line = i.strip().split('\t')
    #print(line[5])
    if line[5] == strand:
        #print(line[5])
        to_file.write(i)

In [20]:
to_file.close()
df.close()

In [26]:
import numpy as np
import pandas as pd
from sys import argv

In [27]:
filename_bed    = 'sort_bed_plus.bed'
filename_rsites = 'restr_frag_shortname.bed'
strand  = '+'
outfile = 'distance_restr.txt'

try:
    rsites = pd.read_csv(filename_rsites, header=None, sep='\t')
except pd.errors.EmptyDataError:
    rsites = pd.DataFrame(columns=np.arange(6))
rsites.columns = ['chrom', 'start', 'end', 'name', '_', 'strand']
rsites = rsites.loc[rsites.strand == strand, :].sort_values(['chrom', 'start'])
rsites_grouped = rsites.groupby('chrom')

try:
    bed_file = pd.read_csv(filename_bed, sep='\t', header=None)
except pd.errors.EmptyDataError:
    bed_file = pd.DataFrame(columns=np.arange(7))
bed_file.columns = ['chrom', 'start', 'end', 'read_id', 'q', 'strand', 'cigar']

chs = bed_file.chrom.values.astype(int)
ids = bed_file.read_id.values.astype(str)
bgn = bed_file.start.values.astype(int)
end = bed_file.end.values.astype(int)
l = len(chs)



In [38]:
dct = {}
for k in ['start_left', 'start_right', 'end_left', 'end_right']:
    dct[k] = np.full(l, -1).astype(int)

for ch in rsites_grouped.groups.keys():
    rs = np.concatenate( [[-1e10], rsites_grouped.get_group(ch)['start'].values, [1e10]] )
    mask = chs == ch

    idx = np.digitize(bgn[mask], rs)
    bgns = rs[idx - 1]
    print('rest_start ' + str(rs[idx - 1][0]))
    print('read_start ' + str(bgn[mask][0]))
    ends = rs[idx]
    print('rest_end  ' + str(rs[idx][0]))
    dct['start_left'][mask] = bgns - bgn[mask]
    dct['start_right'][mask] = ends - bgn[mask]  

    idx = np.digitize(end[mask], rs)
    bgns = rs[idx - 1]
    ends = rs[idx]
    dct['end_left'][mask] = bgns - end[mask] 
    dct['end_right'][mask] = ends - end[mask] 

with open(outfile, 'w') as outf:
    for i in range(l):
        outf.write(
            ids[i] + " " + \
            str(dct['start_left'][i]) + " " + \
            str(dct['start_right'][i]) + " " + \
            str(dct['end_left'][i]) + " " + \
            str(dct['end_right'][i]) + "\n")

rest_start 165636239.0
read_start 165636450
rest_end  165636470.0
rest_start 65343838.0
read_start 65343861
rest_end  65343881.0
rest_start 49817226.0
read_start 49817227
rest_end  49817247.0


Код для проверки леммы о том, что один конец рида должен быть далеко от одного сайта рестрикции и близко к другому сайту рестрикции

In [53]:
to_file = open('sorted_reads', 'w')

In [55]:
my_sample = open('sample_filter.txt', 'r') 
for i in my_sample:
    line = i.strip().split(' ')
    x = float(line[1])
    z = float(line[3])
    w = float(line[4])
    y = float(line[2])
    #print(x, z)
    if x == 0:
        print('hi')
        if y > 100 and z > -50 and w > 100:
            to_file.write(i) 
    elif z == -4:
        print(i)
        if w > 100 and x < -100 and y < 50:
            print("added ")
            to_file.write(i)
    else:
        print(i)

D00795:30:CA2UTANXX:1:1107:1327:1890 -370 20 -4 297

added 
D00795:30:CA2UTANXX:1:1107:1327:1890 -44 20 -4 31

D00795:30:CA2UTANXX:1:1107:1327:1890 -79 20 -4 156

D00795:30:CA2UTANXX:1:1107:1327:1890 -44 20 -4 31

D00795:30:CA2UTANXX:1:1107:1271:1955 -15 20 -4 36

D00795:30:CA2UTANXX:1:1107:1271:1955 -44 20 -4 146

D00795:30:CA2UTANXX:1:1107:1350:1994 -156 20 -4 76

D00795:30:CA2UTANXX:1:1107:1350:1994 -28 20 -4 40

D00795:30:CA2UTANXX:1:1107:1350:1994 -1 20 -4 76

D00795:30:CA2UTANXX:1:1107:1350:1994 -111 20 -4 75

D00795:30:CA2UTANXX:1:1107:1350:1994 -455 20 -4 76

D00795:30:CA2UTANXX:1:1107:1350:1994 -63 20 -4 74

D00795:30:CA2UTANXX:1:1107:1350:1994 -1 20 -4 76

D00795:30:CA2UTANXX:1:1107:1350:1994 -1 20 -4 74

D00795:30:CA2UTANXX:1:1107:1350:1994 -29 20 -4 76

D00795:30:CA2UTANXX:1:1107:1350:1994 -25 20 -4 26

D00795:30:CA2UTANXX:1:1107:1350:1994 -241 20 -4 74

D00795:30:CA2UTANXX:1:1107:1350:1994 -1 20 -4 121

D00795:30:CA2UTANXX:1:1107:1350:1994 -47 20 -4 76

D00795:30:CA2UTANXX

In [51]:
to_file.close()